# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

#### Gathering Data 

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib
import matplotlib.pyplot as plt
# % matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

#### Assess Data

In [2]:
print(portfolio.head())
print(portfolio.shape)

                       channels  difficulty  duration  \
0       [email, mobile, social]          10         7   
1  [web, email, mobile, social]          10         5   
2          [web, email, mobile]           0         4   
3          [web, email, mobile]           5         7   
4                  [web, email]          20        10   

                                 id     offer_type  reward  
0  ae264e3637204a6fb9bb56bc8210ddfd           bogo      10  
1  4d5c57ea9a6940dd891ad53e9dbe8da0           bogo      10  
2  3f207df678b143eea3cee63160fa8bed  informational       0  
3  9b98b8c7a33c4b65b9aebfe6a799e6d9           bogo       5  
4  0b1e1539f2cc45b7b9fa7c272da2e1d7       discount       5  
(10, 6)


In [3]:
portfolio.dtypes

channels      object
difficulty     int64
duration       int64
id            object
offer_type    object
reward         int64
dtype: object

In [4]:
print(profile.head())
print(profile.shape)

   age  became_member_on gender                                id    income
0  118          20170212   None  68be06ca386d4c31939f3a4f0e3dd783       NaN
1   55          20170715      F  0610b486422d4921ae7d2bf64640c50b  112000.0
2  118          20180712   None  38fe809add3b4fcf9315a9694bb96ff5       NaN
3   75          20170509      F  78afa995795e4d85b5d9ceeca43f5fef  100000.0
4  118          20170804   None  a03223e636434f42ac4c3df47e8bac43       NaN
(17000, 5)


In [5]:
profile.dtypes

age                   int64
became_member_on      int64
gender               object
id                   object
income              float64
dtype: object

In [6]:
profile.isna().sum(axis=0)

age                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64

In [7]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [8]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [9]:
print(transcript.shape)

(306534, 4)


In [10]:
transcript.isna().sum(axis=0)

event     0
person    0
time      0
value     0
dtype: int64

In [11]:
transcript.dtypes

event     object
person    object
time       int64
value     object
dtype: object

### Data Preparation/ Exploration

#### Data Cleaning

In [12]:
# Convert int to datetime
profile.became_member_on = pd.to_datetime(profile.became_member_on, format = '%Y%m%d')

In [13]:
profile.head()

,age,became_member_on,gender,id,income
0,118,2017-02-12,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,2017-07-15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,2018-07-12,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,2017-05-09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,2017-08-04,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [14]:
profile.dtypes

age                          int64
became_member_on    datetime64[ns]
gender                      object
id                          object
income                     float64
dtype: object

In [15]:
# Remove Outliers that potentially have incorrect ages
Q1 = profile.age.quantile(0.25)
Q3 = profile.age.quantile(0.75)
IQR = Q3 - Q1
print(IQR)
profile_1 = profile.loc[(profile['age']  > Q1 - 1.5 * IQR) & (profile['age']  < Q3 + 1.5 * IQR)]


28.0


In [16]:
# One hot encoding

def one_hot_encoding(df, cl):
    for column in cl:
        column_dummies = pd.get_dummies(df[column])
        df = df.drop(column, axis = 1)
        df = df.join(column_dummies)
    return df

In [17]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [18]:
# One hot encode
columns = ['offer_type']
portfolio_encode = one_hot_encoding(portfolio, columns)
portfolio_encode.head()

,channels,difficulty,duration,id,reward,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0


In [19]:
portfolio_encode['web'] = portfolio_encode['channels'].apply(lambda x: 1 if 'web' in x else 0)
portfolio_encode['email'] = portfolio_encode['channels'].apply(lambda x: 1 if 'email' in x else 0)
portfolio_encode['mobile'] = portfolio_encode['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
portfolio_encode['social'] = portfolio_encode['channels'].apply(lambda x: 1 if 'social' in x else 0)

In [20]:
portfolio_encode = portfolio_encode.drop('channels', axis = 1)


In [21]:
portfolio_encode.describe()

,difficulty,duration,reward,bogo,discount,informational,web,email,mobile,social
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.000000,10.000000
mean,7.700000,6.500000,4.200000,0.400000,0.400000,0.200000,0.800000,1.0,0.900000,0.600000
std,5.831905,2.321398,3.583915,0.516398,0.516398,0.421637,0.421637,0.0,0.316228,0.516398
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
25%,5.000000,5.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.000000
50%,8.500000,7.000000,4.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,1.000000
75%,10.000000,7.000000,5.000000,1.000000,1.000000,0.000000,1.000000,1.0,1.000000,1.000000
max,20.000000,10.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000


In [22]:
# Characteristics about the data - email is all one so does not have any predictive attributes
portfolio_encode = portfolio_encode.drop('email', axis = 1)


In [23]:
portfolio_encode.head()

,difficulty,duration,id,reward,bogo,discount,informational,web,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,0,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0


In [24]:
# One hot encode transcript
columns_transcript = ['event']
transcript_encode = one_hot_encoding(transcript, columns_transcript)
transcript_encode.head()

,person,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [25]:
def display_keys(x):
    if x.keys() not in all_keys:
        all_keys.append(x.keys())

all_keys = []
transcript_encode['value'].apply(lambda x: display_keys(x))
print(all_keys)

[dict_keys(['offer id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [26]:
def check_key(x, column, column1):
#       '''
#     data cleaning for transcript dataframe. Splits the values  based on parameters
    
#     INPUT:
#     columns to strip for new column
    
#     OUTPUT:
#     new column containing expanded data
    
#     '''
    if column in x.keys():
        return x[column]
    if column1 in x.keys():
        return x[column1]

In [27]:
# Split offers into new column
offer_id = transcript_encode['value'].apply(lambda x: check_key(x, 'offer_id','offer id'))
transcript_encode['offer_id'] = offer_id

In [28]:
# split amount into new column
amount = transcript_encode['value'].apply(lambda x: check_key(x, 'amount','amount'))
transcript_encode['amount'] = amount

In [29]:
# split reward into new column
reward = transcript_encode['value'].apply(lambda x: check_key(x, 'reward','reward'))
transcript_encode['reward'] = reward

In [30]:
transcript_encode = transcript_encode.drop('value', axis = 1)

# Fill all of the NaN items with 0
transcript_encode.reward.fillna(0 , inplace=True)
transcript_encode.amount.fillna(0 , inplace=True)

In [31]:
# Remove all of the N/A offers
transcript_encode = transcript_encode[transcript_encode['offer_id'].notna()]

In [32]:
transcript_encode.head()

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0


In [33]:
transcript_encode = transcript_encode.rename(columns={"reward": "reward_received"})

In [34]:
# Find duration from start to finish

In [35]:
transcript_encode_temp = transcript_encode.loc[transcript_encode['offer completed'] == 1].filter(['person','offer_id','time'])

In [36]:
transcript_encode = transcript_encode.merge(transcript_encode_temp,how='left',on=['offer_id','person'])

In [37]:
transcript_encode = transcript_encode.rename(columns={"time_x": "time"})

In [38]:
transcript_encode['duration_to_complete'] = (transcript_encode['time_y'] - transcript_encode['time'])/24

In [39]:
transcript_encode = transcript_encode.drop('time_y', axis = 1)

In [46]:
# Merge Portfolio with Transcript
transcript_portfolio=transcript_encode.merge(portfolio_encode,how='left',left_on='offer_id',right_on ='id')


In [47]:
transcript_portfolio = transcript_portfolio.drop_duplicates(subset=None, keep="first", inplace=False)

In [50]:
transcript_portfolio1.head()

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward_received,duration_to_complete,difficulty,duration,id,reward,bogo,discount,informational,web,mobile,social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,5.5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0,NaN,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0.0,NaN,10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0,NaN,10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,NaN,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1


In [48]:
transcript_portfolio1 = transcript_portfolio.loc[((transcript_portfolio.duration_to_complete < transcript_portfolio.duration) & (transcript_portfolio.duration_to_complete >= 0))| transcript_portfolio.duration_to_complete.isnull() == True]

In [ ]:
# Create sum of total of completed offers by person
# transcript_portfolio1['completed_offers_by_person'] = transcript_portfolio1['offer completed'].groupby(transcript_portfolio['person']).transform('sum')

In [ ]:
transcript_portfolio1_temp = transcript_portfolio1.filter(['time','offer viewed','person','offer_id'])

In [51]:
transcript_portfolio1.loc[(transcript_portfolio1.offer_id == '0b1e1539f2cc45b7b9fa7c272da2e1d7') & (transcript_portfolio1.person== 'd53717f5400c4e84affdaeda9dd926b3')]

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward_received,duration_to_complete,difficulty,duration,id,reward,bogo,discount,informational,web,mobile,social
15,d53717f5400c4e84affdaeda9dd926b3,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0,1.25,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
23944,d53717f5400c4e84affdaeda9dd926b3,30,0,0,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0,0.00,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
23946,d53717f5400c4e84affdaeda9dd926b3,30,1,0,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,5.0,0.00,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
159924,d53717f5400c4e84affdaeda9dd926b3,576,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0,2.00,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
189903,d53717f5400c4e84affdaeda9dd926b3,624,1,0,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,5.0,0.00,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0


In [67]:
transcript_portfolio1_temp = transcript_portfolio1.filter(['time','person','offer_id']).loc[transcript_portfolio1['offer viewed'] == 1]

In [73]:
transcript_portfolio2 = transcript_portfolio1.merge(transcript_portfolio1_temp,how='left',on=['offer_id','person'])

In [74]:
transcript_portfolio2['Viewed_Duration'] = (transcript_portfolio2['time_y'] - transcript_portfolio2['time_x'])/24

In [ ]:
transcript_portfolio2

In [77]:
transcript_portfolio2 = transcript_portfolio2.loc[((transcript_portfolio2.Viewed_Duration < transcript_portfolio2.duration))| transcript_portfolio2.Viewed_Duration.isnull() == True]

In [76]:
transcript_portfolio2 = transcript_portfolio2.drop_duplicates(subset=None, keep="first", inplace=False)

In [97]:
transcript_portfolio2.to_csv('temp.csv')

In [95]:
transcript_portfolio2.loc[(transcript_portfolio2.offer_id == 'a03223e636434f42ac4c3df47e8bac43') & (transcript_portfolio2.person== 'd53717f5400c4e84affdaeda9dd926b3')]

,person,time_x,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward_received,duration_to_complete,...,id,reward,bogo,discount,informational,web,mobile,social,time_y,Viewed_Duration



Finding people who are converting based on completing offer recevied

1. People who are influenced and successfully convert - completed offer:


 `offer received` -> `offer viewed`  -> `offer completed` 

2. People who received but did not complete offer - incomplete offer:

 `offer received` -> `offer viewed`
 `offer received`
 
 offer needs to be completed within the offer timeframe

In [105]:
transcript_portfolio3 = transcript_portfolio2.loc[transcript_portfolio2['offer received'] == 1]

In [87]:
transcript_portfolio3.head()

,person,time_x,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward_received,duration_to_complete,...,id,reward,bogo,discount,informational,web,mobile,social,time_y,Viewed_Duration
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,5.5,...,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0,6.0,0.25
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0,NaN,...,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0,6.0,0.25
3,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0.0,NaN,...,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,0,18.0,0.75
4,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0,NaN,...,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1,12.0,0.50
6,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,NaN,...,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1,84.0,3.50


In [106]:
# If the duration to complete is N/A then it was not completed..
# if viewed_duration is negative then this was not viewed during this transaction

transcript_portfolio3['offer viewed'] = np.where(transcript_portfolio3['Viewed_Duration']< 0, 0,1)
transcript_portfolio3['offer completed'] = np.where(transcript_portfolio3['duration_to_complete'].isnull() == True, 0,1)
transcript_portfolio3['reward_received'] = np.where(transcript_portfolio3['offer completed'] == 1, transcript_portfolio3['reward'] ,0)

C:\Users\jlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\jlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\jlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [107]:
transcript_portfolio3 = transcript_portfolio3.drop(['time_y','time_x','id'], axis = 1)

In [108]:
transcript_portfolio3.head()

,person,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward_received,duration_to_complete,difficulty,duration,reward,bogo,discount,informational,web,mobile,social,Viewed_Duration
0,78afa995795e4d85b5d9ceeca43f5fef,1,1,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5,5.5,5,7,5,1,0,0,1,1,0,0.25
1,a03223e636434f42ac4c3df47e8bac43,0,1,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0,NaN,20,10,5,0,1,0,1,0,0,0.25
3,e2127556f4f64592b11af22de27a7932,0,1,1,0,2906b810c7d4411798c6938adc9daaa5,0.0,0,NaN,10,7,2,0,1,0,1,1,0,0.75
4,8ec6ce2a7e7949b1bf142def7d0e0586,0,1,1,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0,NaN,10,10,2,0,1,0,1,1,1,0.50
6,68617ca6246f4fbc85e91a2a49552598,0,1,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0,NaN,10,5,10,1,0,0,1,1,1,3.50


#### Encode Profile 

In [ ]:

# One hot encode
columns_profile = ['gender']
profile_encode = one_hot_encoding(profile, columns_profile)
# Drop N/A
profile_encode.dropna(inplace=True)

profile_encode.head()

### Data Analysis 

#### Vizualizations 

In [ ]:
import seaborn as sns

In [ ]:
def age_group(row):
    if row["age"] < 13:
        return "age 12 years old"
    elif row["age"] < 18:
        return "12-17"
    elif row["age"] < 25:
        return "18-24"
    elif row["age"] < 35:
        return "25-34"
    elif row["age"] < 45:
        return "35-44"
    elif row["age"] < 55:
        return "45-54"
    elif row["age"] < 65:
        return "55-64"
    elif row["age"] < 75:
        return "65-74"
    else:
        return "75+"

In [ ]:
def income_group(row):
    if row["income"] < 20000:
        return "Less than 20,000"
    elif row["income"] < 35000:
        return "20,000 to 34,999"
    elif row["income"] < 50000:
        return "35,000 to 49,999"
    elif row["income"] < 75000:
        return "50,000 to 74,999"
    elif row["income"] < 100000:
        return "75,000 to 99,999"
    else:
        return "100K+"
    



In [ ]:
profile_encode['Age_Group'] = profile_encode.apply(age_group, axis=1)


In [ ]:
profile_encode['Income_Group'] = profile_encode.apply(income_group, axis=1)


##### What is the distribution of Ages and Income

In [ ]:
profile_encode1 = profile_encode.rename(columns={"id": "count"})
profile_encode_age = profile_encode1.groupby('Age_Group').count()
profile_encode_age.reset_index(inplace=True)
sns.barplot(x='Age_Group', y='count', data=profile_encode_age);

In [ ]:


profile_encode1 = profile_encode.rename(columns={"id": "count"})
profile_encode_income = profile_encode1.groupby('Income_Group').count()
profile_encode_income.reset_index(inplace=True)
# plt.rcParams["axes.labelsize"] = 12
# plt.rc('xticks',fontsize=5) 
plt.figure(figsize=(20,5))
sns.barplot(x='Income_Group', y='count', data=profile_encode_income);


In [ ]:
profile_encode_income_age = profile_encode1.groupby(['Income_Group','Age_Group'],as_index=False).count()
plt.figure(figsize=(30,5))

sns.factorplot(x='Age_Group', y='count', hue='Income_Group', data=profile_encode_income_age, kind='bar')


##### What is the distirbution of Gender 

In [ ]:
profile1 = profile.rename(columns={"id": "count"})

profile_encode_gender = profile1.groupby('gender').count()
profile_encode_gender.reset_index(inplace=True)
sns.barplot(x='gender', y='count', data=profile_encode_gender);

### Modeling Methodology 

In [ ]:
profile_encode.head()

In [ ]:
transcript_encode.head()